<a href="https://colab.research.google.com/github/grniemeyer/IANNwTF_hw2/blob/main/PlayingWithMnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
### Data prep

import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np

# tensorflow = (data) pipeline, factory
# tensorflow tensor like array


(train_ds,test_ds), ds_info = tfds.load('mnist', split=['train', 'test'], as_supervised=True, with_info=True)
# print(ds_info)
# tfds.show_examples(train_ds, ds_info)


# remember order for exam!!
# #ds = train_ds.map(lambda feature_dict: (feature_dict["image"], feature_dict["label"])) # ectract images and features
# ds = train_ds.map(lambda image, label: (tf.reshape(image, (-1,)), label)) # flatten images, pass on labels
# ds = ds.map(lambda image, label: ((tf.cast(image, tf.float32) / 128.0) -1, label)) # normalize images and pass on
# ds = ds.map(lambda image, label: (image, tf.one_hot(label, depth=10))) # one hottify
# ds = ds.shuffle(1024).batch(4) # create small buffer(9 pieces of choc bzw. 1024) in between for shuffling then batch; tf's batch operation takes care of the remainder shape
# train_ds = ds.prefetch(4) # parralelisation: 2nd buffer that has always shuffled (prefetched) batches availabe. Shuffling & prefetching takes lot of memory



# or: Combine reshaping, normalization, and one-hot encoding into a single mapping function
def preprocess_image(image, label):
    image = tf.reshape(image, (-1,))
    image = (tf.cast(image, tf.float32) / 128.0) - 1
    image = tf.expand_dims(image, axis=0)  # debugging: Add a batch dimension
    return image, label


# Apply the preprocessing to your datasets
train_ds = train_ds.map(preprocess_image)
test_ds = test_ds.map(preprocess_image)

train_ds = train_ds.map(lambda image, label: ((tf.cast(image, tf.float32) / 128.0) - 1, tf.one_hot(label, depth=10)))
test_ds = test_ds.map(lambda image, label: ((tf.cast(image, tf.float32) / 128.0) - 1, tf.one_hot(label, depth=10)))



for x, y in train_ds.take(1):
  print(x, y)

# • How many training/test images are there?
# - 'test' 10,000, train' 60,000
# • What’s the image shape?
# - (28, 28, 1)
# • What range are pixel values in?
# - In grayscale images, a pixel value of 0 represents black, and 255 represents white

tf.Tensor(
[[-1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125
  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125
  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125
  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125
  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125
  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125
  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125
  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125
  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125
  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125
  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125
  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125
  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125  -1.0078125
  -1.0078125  -1.0078125  -1.0078125  -1

In [12]:
### 2.3 Building a DNN with TF
from tensorflow.keras.layers import Dense

class MyDeepNN(tf.keras.Model):
  def __init__(self):
    #self.image = image
    super(MyDeepNN, self).__init__()
    self.dense1 = tf.keras.layers.Dense(256, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(256, activation=tf.nn.relu)
    self.out = tf.keras.layers.Dense(10, activation=tf.nn.softmax)

  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    y = self.out(x)
    return y


# class MyDeepNN(tf.keras.Model):
#     def __init__(self):
#         super(MyDeepNN, self).__init__()
#         self.dense1 = Dense(256, activation=tf.nn.relu)
#         self.dense2 = Dense(256, activation=tf.nn.relu)
#         self.out = Dense(10, activation=tf.nn.softmax)

#     def call(self, inputs):
#         x = self.dense1(inputs)
#         x = self.dense2(x)
#         y = self.out(x)
#         return y





In [13]:
### 2.4 Training the Network


# define hyperparameters
nr_epochs = 10
learning_rate = 0.1

# initialize stuff
mymodel = MyDeepNN()
cross_entropy_loss = tf.keras.losses.CategoricalCrossentropy()
opti = tf.keras.optimizers.SGD(learning_rate)

# lists (for viualization later)
train_losses = []
test_losses = []
test_accuracies = []



def test(model, test_ds, loss_function):
  test_accuracy_aggregator = []
  test_loss_aggregator = []

  for (image, target) in test_ds.take(5): # adjusted to 5 for now
    prediction = model(image)
    # debugging 5.0: Assuming target is one-hot encoded, use tf.reduce_sum along the axis to get the correct shape
    target = tf.reduce_sum(target, axis=0)
    prediction = tf.squeeze(prediction, axis=0)  # debugging 2.0: Remove the batch dimension
    # Print the shapes for debugging
    print("Target shape:", target.shape)
    print("Prediction shape:", prediction.shape)
    # debugging 6.0: Reshape the target to match the shape of prediction
    target = tf.reshape(target, prediction.shape)
    sample_test_loss = loss_function(target, prediction)
    # # debugging 3.0: Check the dimensions before applying np.argmax
    # if len(target.shape) == 1:
    #   sample_test_accuracy = np.argmax(target) == np.argmax(prediction)
    # else:
    #   sample_test_accuracy = np.argmax(target, axis=1) == np.argmax(prediction, axis=1)

    sample_test_accuracy = np.argmax(target, axis=1) == np.argmax(prediction, axis=1)
    sample_test_accuracy = np.mean(sample_test_accuracy)
    test_loss_aggregator.append(sample_test_loss.numpy())
    test_accuracy_aggregator.append(np.mean(sample_test_accuracy))# double averaging??

  test_loss = tf.reduce_mean(test_loss_aggregator)
  test_accuracy = tf.reduce_mean(test_accuracy_aggregator)

  return test_loss, test_accuracy


# test before sending through NN, for comparison afterwards
test_loss, test_accuracy = test(mymodel, test_ds, cross_entropy_loss)
test_losses.append(test_loss)
test_accuracies.append(test_accuracy)

# test with train_ds before training NN
train_loss, _ = test(mymodel, train_ds, cross_entropy_loss)
train_losses.append(train_loss)


# tests on 1 input-target pair
def train_step(model, input_data, target, loss_function, optimizer):
  with tf.GradientTape() as tape:
    prediction = model(input_data)
    loss = loss_function(target, prediction)
  gradients = tape.gradient(loss, model.trainable_variables) # trainable_variables = weights?
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss



for epoch in range(nr_epochs):
#  print(f"epoch: {str(epoch)} with accuracy {test_accuracies[-1]}") # starting with last element in test_accuracies

  # training
  epoch_losses = []
  for input, label in train_ds:
    train_loss = train_step(mymodel, input, label, cross_entropy_loss, opti)
    epoch_losses.append(train_loss)

  # track training loss
  train_losses.append(tf.reduce_mean(epoch_losses)) #take the mean

  # testing (track test loss and test accuracy)
  test_loss, test_accuracy = test(mymodel,test_ds, cross_entropy_loss)
  test_losses.append(test_loss)
  test_accuracies.append(test_accuracy)


Target shape: ()
Prediction shape: (10,)


InvalidArgumentError: ignored

In [ ]:
# Training

EPOCHS = 10

model = MLP_Model(layer_sizes=[256, 256])
cce = tf.keras.CategoricalCrossEntropy()
optimizer = tf.keras.optimizers.legacy.SGD() # stochastic gradient as optimizer
ds = ds # does nothing, just as reminder from above

for epoch in range(EPOCHS):
  losses = []

  for x, target in ds:
    with tf.GradientTape() as tape: # context manager
      pred = model(x)
      loss = cce(target, pred)
    gradients = tape.gradient(loss, model.varibles) # calculate gradient outside of GradientTape, otherwis it calculates gradient of gradient (memory!!, rare exceptions)
    optimizer.appy_gradients(zip(gradients, model.variables))
    losses.append(loss.numpy())
  print(np.mean(losses))

In [34]:
# Model Creation via Subclassing from tf.keras.Model

# #inherit from keras.Model
# class MLP_Model(self, layer_size):
#   def __init__(self, layer_size, output_size=10):
#     super().__init__()
#     self.layers = []
#     #layer_size e.g.[256, 256]

#     for layer_size in layer_sizes:
#       new_layer = tf.keras.layers.Dense(units=layer_size, activation="sigmoid")
#       self.layers.append(new_layer)
#     self.output_layer = tf.keras.layers.Dense(units=output_size, activation="softmax") # output layer ist deshalb specified seperately bc we need different activation function

#   def call(self, x):
#     for layer in self.layers:
#       x = layer(x) # call layer on input
#     y = self.output.layer(x)
#     return y

ModuleNotFoundError: ignored